# Logistic Regression with Stochastic Gradient Descent

This notebook explored the process of implementing a logistic regression model using Stochastic Gradient Descent (SGD) from scratch in Python. The [data](https://www.kaggle.com/lucidlenn/sloan-digital-sky-survey) used for this tutorial is by the Sloan Digital Sky Survey. Thanks to [Lennart Grosser](https://www.kaggle.com/lucidlenn) for providing this data set.
<br><br>

While the aim of the notebook is to build a logisitc regression model, the dataset is known to have 3 different classes. TThe notebook will only focus on creating a logistic regression for a binary class problem. 

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import math

In [ ]:
data = pd.read_csv('../input/Skyserver_SQL2_27_2018 6_51_39 PM.csv')

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
data.describe()

Except for the 'rerun' column, there are no missing values in the dataset. Except the 'dec' and 'spaceobjid', the mean and medians (50 percentile) of all features are quite similar.

## Data Wrangling

Having had understood the data spread and features, we can prepare the data needed to develop the model.

Since we don't have to deal with missing values except for the 'rerun' column, lets find out the number of unique values in each column.

In [ ]:
for cols in data.columns:
    print(cols,':',data[cols].nunique())

As provided in the data description, 'class' is the only feature that isn't continous which has to be predicted. While the dataset has 10000 datapoints, it can be noticed that there are columns like 'objid' and 'rerun' that only have one value. These columns can be removed. The 'spaceobjid' feature is an object identifier with fairly large values. Similarly, the 'plate' feature is a number identifying the plate used in the telescope. These feature being IDs cannot be scaled and thus it can be removed as well. 

Note: A computer with a GPU acess can be used to create one-hot encoders for the plate numbers and object identifiers, but since this is a simple tutorial, these values will be ignored.

In [ ]:
# Delete columns
data.drop(['objid', 'rerun', 'specobjid', 'plate'], axis=1, inplace=True)

Analyse the correlation among the features in the dataset.

In [ ]:
data.corr()

The correlation matrix above shows that features g, r , i , z are strongly correlated with each other. While the deature u is only correlated to g, it seems independent from the others. Since we are dealing with a simple logisitic regression model, we shall eliminate the correlation by removing columns g, r and i. Feature u is not highly correlated with other features and among g, r, i, z, the feature z is least correlated to u. Thus we retain these two columns among the 5 bands of the telescope.

Feature 'mjd' is also highly correlated with 'plate', although they do not have any meaningful data in common. Since mjd is the 'Modified Julian Date' that identifies the date the SDSS image was taken, we can eliminate it along with the others while retaining the 'plate' feature. 

In [ ]:
data.drop(['g', 'r', 'i', 'mjd'], axis=1, inplace=True)

Now analyse all the reamining columns. We start with the features that describe a field within an image taken by the SDSS.

In [ ]:
# 1. 'run' column identifies specific scan 
data['run'].value_counts()

In [ ]:
# 2. 'camcol' column identifies scanline with 'run' 
data['camcol'].value_counts()

Since this feature is of an integer type and only has 6 values, it can be treated as a categorical variable. Dummy variables can be created to accomodate all the factors in this features.

In [ ]:
dummy_df = pd.get_dummies(data['camcol'])
new_data = pd.concat([data, dummy_df], axis=1)
new_data.drop('camcol', axis=1, inplace=True)
new_data.head()

In [ ]:
# Analyse class column
data['class'].value_counts(normalize=True)*100

Almost 50% of the dataset contains data identifying galaxies while 41.5% are stars with the remaining 8% as quasars. We shall be eliminating quasars ('QSO') and develop a binary classification model.

In [ ]:
# Create subset of data
data1 = new_data[new_data['class']!='QSO']

Analyse the mean values of the features w.r.t. the 'class' factor.

In [ ]:
data1.groupby('class').mean()

While most of the feature mean values are similar for either class, there are a few with noteworthy differences. The right ascension (ra), declination (dec) and 'redshift' have higher means for the GALAXY class, suggesting that higher values of these 3 features indicate the object to be a galaxy. This can later be confirmed with the feature importance using scikit's sklearn toolbox.

## Developing the Model

The data has been cleaned and prepared to now develop a model. In short, the logisitic regression model calculates the probabilities of a data point belonging to a certain class. The model first calculates a linear model (a.k.a. linear regression) and then uses the sigmoid function to calculate the probabilies. The final step is using a decision boundary to decide which class the data point belongs when compared to the decision bounday values. 

To create and test the model, we will need train and test data sets.

In [ ]:
from sklearn.model_selection import train_test_split
X = data1.drop('class', axis=1)
y = data1['class']
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.25, random_state=1551)

In [ ]:
X_train.head()

Notice the values of the features above. All columns excluding the dummy variables for the 'camcol' feature are of varying scales. In order for the logistic regression model to calculate the distances between the points, the data should be across a similar range. Generating a model with varying data scales result in an unstable prediction. To avoid that, we use MinMax scaler (among others) to scale the features.

In [ ]:
from sklearn.preprocessing import MinMaxScaler
train_scale = MinMaxScaler().fit(X_train[['ra', 'dec', 'u', 'z', 'run', 'field', 'redshift', 'fiberid']])
train_trans_data = train_scale.transform(X_train[['ra', 'dec', 'u', 'z', 'run', 'field', 'redshift', 'fiberid']])
X_train = np.column_stack([train_trans_data,X_train[[1,2,3,4,5,6]]])

# scale the test set as well
test_trans_data = train_scale.transform(X_test[['ra', 'dec', 'u', 'z', 'run', 'field', 'redshift', 'fiberid']])
X_test = np.column_stack([test_trans_data,X_test[[1,2,3,4,5,6]]])

Use Label Encoder to convert the classes ('GALAXY', 'STAR') to numeric values to be used in the model.

In [ ]:
from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()
y_train = lb.fit_transform(y_train)
y_test = lb.transform(y_test)

As mentioned above, the logistic regression uses a sigmoid function to calculate the probabilities of the class from a linear model. It can be written as:
>                 h(x) = g(z)
                where z = bias + theta_1 * X_1 + theta_2 * X_2 + ... + theta_n * X_n 
                and g(z) = 1/(1+e^−z)
                
Let's break down the functions above. h(x) is the probability that the output is 1. In the g(z) function, z denotes the linear regression model that outputs a real number as a predicted value on a straight line. Since a straight line cannot fit a binary response (0,1), it has to be encoded between 0 and 1 as probabilities. This probability will be mapped to predict the data point belonging to the positive class, i.e. 1 [1]. The calculation of this probability is done by the sigmoid function, denoted as g(z). The function g(z) maps a real number between 0, 1 using the sigmoid calculation [2]. If the probability is greater than 0.5, the data point can be classified as the positive class (1), else it can be classified as the negative class (0). Here, 0.5 is the a decision boundary, which can be subject to change for different hypotesis and datasets. We shall maintain 0.5 as our decision boundary.
 
The first part to be calculated is the linear regression model (z). The bias is the intercept and the theta valuea are the coefficients for each feature in the data set. The outcome of the function g(z) is a probability of a data point belonging to a certain class.

In order to classify a data point accurately, we need to minimize the cost function which is:
>                J = -1/m * sum(y * log(h(x)) + (1-y) * log(1-h(x)))

Here, the first part of the funtion (y * log(h(x)) will be 0 when y=0 and the second part of the function comes into effect. Conversely, the second part of the function ((1-y) * log(1-h(x))) equates to 0 when y=1, thus activating only the first part. 

We are using Stochastic Gradient Descent (SGD) to find optimal values of the coefficients. That can be done using the derivative of the cost function, the coefficients (theta) values of the variables can be adjusted using a learning rate alpha. The coefficient are updated as follows:
>                theta = theta - alpha/m * sum (h(x) - y) * X

Using a set (large) number of iterations and fairly small value of theta, the cost function will converge at a local minima, thus producing coefficients that predict values as close to the dependent (y) values as possible.

Now that we have the process in place, we can develop the model. First we prepare the variables needed for the model.

In [ ]:
X = np.column_stack(([1]*X_train.shape[0], X_train)) # add column of 1s as bias 
m,n = X.shape # get rows and columns
y = y_train
theta = [1]*n # initiate array of coefficents
iterations = 1800
alpha = 0.5 # alpha value to update theta
cost = []
t_temp = []

In [ ]:
# Sigmoid function
def sigmoid(val):
    y = 1/(1 + np.exp(-val)) # math.exp() calculates e to the power of -(theta * X)
    return y

In [ ]:
for i in range(0,iterations):
    val = np.matmul(X,theta) # calculate linear regression 
    y_prob = sigmoid(val) # calculate probability 
    
    J = -1/m * sum((y*np.log(y_prob)) + (1- y) * np.log(1-y_prob))
    cost.append(J) # append cost function values to list
    
    # Update theta values as a gradient of the cost function
    for t in range(0,n):
        temp = round(theta[t] - alpha/m * sum((y_prob - y)*X[:,t]),4)
        t_temp.append(temp)
    
    theta = []
    theta = t_temp
    t_temp = []

In [ ]:
print('The final Theta values are: ',theta)

In [ ]:
plt.figure(figsize=(10,8))
plt.title('Cost Function Slope')
plt.plot(cost)
plt.xlabel('Number of Iterations')
plt.ylabel('Error Values')
None

The plot above shows that the error/cost function values sharply decline in the first few iterations and proceed to reduce slowly over the rest of the iterations. it can be assumed that the local minima was achieved at the 1800th iteration. This shows that the logistic regression model has converged with optimum values for the coefficients. Using these new values, we can predict the values for the test set and check the goodness of fit using model metrics.
<br><br>
You can experiment by increasing the number of iterations and increase the value of alpha (so it takes larger steps are reaching the minima) if you'd like. 

In [ ]:
# Prepare the data
X_test1 = np.column_stack(([1]*X_test.shape[0],X_test)) # add bias column to X_test

In [ ]:
# Predict values 
y_vals = np.matmul(X_test1,theta) # Multiply theta with X_test
y_prob = sigmoid(y_vals) # calculate probabilities
y_pred = [1 if y>0.5 else 0 for y in y_prob] # Convert probabilities to classes with 0.5 decision boundary

Calculate accuracy and model fit metrics

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_recall_fscore_support
accuracy = accuracy_score(y_test, y_pred,normalize=True)
conf_mat = confusion_matrix(y_test,y_pred)
print("The accuracy of the model is :", round(accuracy,3)*100,"%")
print("Confusion Matrix:\n",conf_mat)

Based on the values of the confusion matrix, we can calculate the precision, recall and F-score to estimate the goodness of the model fit.

In [ ]:
precision, recall, fscore, support = precision_recall_fscore_support(y_test, y_pred, average='binary')
print('Precision = ',round(precision,4),'\nRecall = ', round(recall,4), '\nF-Score = ',round(fscore,4))

While the precision is low, the recall and F-score are comparatively higher. A higher recall value indicates how many of the actual true values were correctly predicted by the model and precision indicates how many of the values predicted true were, in fact, correct. A 0.9178 F-score indicates a good model fit. <br><br>

The final test of the goodness of fit of the model is analysing the AUC plot.

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score
fpr, tpr, _ = roc_curve(y_test,  y_prob)
auc = roc_auc_score(y_test, y_prob)
plt.figure(figsize=(10,8))
plt.plot(fpr,tpr,label="data, auc="+str(round(auc,4)))
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve for Model from Sratch")
plt.legend(loc=4)
plt.show()

The AUC score for the developed model is 0.967. With its close proximity to 1, it can be stated that the model is good at distinguishing between the two classes. 

##  Model using the scikit-learn package

We have created the model from scratch and produced one that classifies the data well. To compare the goodness of fit of our model, we use the predefined scikit-learn package to create a logistic regression model using stochastic gradient descent (SGD) on the same training and test set.

In [ ]:
from sklearn.linear_model import SGDClassifier
classifier = SGDClassifier(loss='log', max_iter=100) # create a classifier using logistic regression using SGD
model = classifier.fit(X_train, y_train) # fit the model on the training data
y_pred = model.predict(X_test) # predict values on test set

In [ ]:
# Deteremine model accuracy and goodness of fit
accuracy = accuracy_score(np.array(y_test),np.array(y_pred),normalize=True)
conf_mat = confusion_matrix(y_test,y_pred)
print("The accuracy of the model is :", round(accuracy,2)*100,"%")
print("Confusion Matrix:\n", conf_mat)

While the accuracy of the model is almost the same as our original model, notice the slight difference in the confusion matrix. The metrics can tell the story futher.

In [ ]:
precision, recall, fscore, support = precision_recall_fscore_support(y_test, y_pred, average='binary')
print('Precision = ',round(precision,4),'\nRecall = ', round(recall,4), '\nF-Score = ',round(fscore,4))

Here we have an almost-perfect recall with the in-built model and a relatively higher score for the precision and f-score than our model, indicating that the in-built model is better than ours.

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score
y_pred_proba = classifier.predict_proba(X_test)[::,1] # get probabilities of the model prediction
fpr, tpr, _ = roc_curve(y_test,  y_pred_proba)
auc = roc_auc_score(y_test, y_pred_proba)
plt.figure(figsize=(10,8))
plt.plot(fpr,tpr,label="data, auc="+str(round(auc,4)))
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve for scikit-learn Model")
plt.legend(loc=4)
plt.show()

The AUC score for the scikit-learn model is slightly higher than our model as well. This goes to show that the in-built model is better at classifying data than our model, although the closeness of the metrics must be noted. While the in-built model is superior, the model developed from scratch is not far behind in proving a good classifier to identify if for the given data, the object in the sky is a Galaxy or a Star. <br><br>
Happy coding and happy star gazing!

# References

[1] https://www.statlearning.com/ <br>
[2] https://www.coursera.org/learn/machine-learning